#Q2.0

Here, i am re-using my functions from the quiz

In [4]:
import numpy as np
from numpy import array
import math

def npPrint(a, f={'float_kind':lambda x: "%.5e" % x}):
    print( np.array2string(a, formatter=f) )

def Q11(f,Df, t0,y0, h):
    y1 = y0 +(h/2)*(f(t0,y0) + f(t0 + h, y0 + h*f(t0, y0)))
    return y1

def forwardEuler(f,Df,  tn,yn, h):
    return yn + h * f(tn,yn)

def computeEocs( herr ):
    eocs = np.zeros(len(herr)-1)
    for i in range(len(herr)-1):
        eocs[i] = math.log(herr[i+1][1]/herr[i][1])/(
        math.log(herr[i+1][0]/herr[i][0]))
    return eocs

def evolve(phi, f,Df, t0,y0, T,N):
    h = T/N
    y = np.zeros((N+1, len(y0)))
    y[0] = y0
    t = np.zeros(N+1)
    t[0] = t0
    for i in range(1,N+1):
        t[i] = t[i-1] + h
    for i in range(1,N+1):
        y[i] = phi(f, Df, t[i-1], y[i-1], h)
    return t,y

#Q2.1

In [15]:
T = 10
lam = 1
N0 = 25
y0 = array([2,-2])
y1 = y0[0]
y2 = y0[1]
count = 0

def f(t, y0):
    global count 
    count +=1
    y1 = y0[0]
    y2 = y0[1]
    return array([y2, y2*(lam - 2*y1)])

def Y(t):
    e = math.e
    return array([(2*e**t)/((2*e**t)-1),
    (-2*e**t)/((4*e**(2*t)) - (4*e**t) + 1)])

herr = np.zeros((10,2))

for i in range(10):
    h = T/(N0*2**i)
    herr[i][0] = h
    t,y = evolve( forwardEuler, f,
                            lambda t,y: y,
                            0,y0, T , N0*2**i)
    errors = np.zeros(len(y))
    for j in range(len(y)):
        errors[j] = np.linalg.norm(y[j]-Y(t[j]))
    herr[i][1] = np.max(errors)

print("Maximum errors for each h value (formatted as [h,error]):\n" , herr)
print("EOCs for each h value:\n", computeEocs(herr))

maxerrs1 = herr[:,1] #this wil be used in q2.3

Maximum errors for each h value (formatted as [h,error]):
 [[4.00000000e-01 1.19752047e+00]
 [2.00000000e-01 3.84901290e-01]
 [1.00000000e-01 1.42446210e-01]
 [5.00000000e-02 6.52549668e-02]
 [2.50000000e-02 3.13565585e-02]
 [1.25000000e-02 1.53834975e-02]
 [6.25000000e-03 7.62053478e-03]
 [3.12500000e-03 3.79267758e-03]
 [1.56250000e-03 1.89196759e-03]
 [7.81250000e-04 9.44898934e-04]]
EOCs for each h value:
 [1.6374899  1.43407127 1.12625762 1.05732049 1.02738368 1.0134194
 1.00667551 1.00332936 1.00165544]


We know that the forward Euler method converges linearly and the EOCs reflect that because they are all close to 1, as we expect.

#Q2.2

The only change from the previous program is that phi is now the Q11 function instead of the forwardEuler function

In [16]:
herr = np.zeros((10,2))

for i in range(10):
    h = T/(N0*2**i)
    herr[i][0] = h
    t,y = evolve( Q11, f,
                            lambda t,y: y,
                            0,y0, T , N0*2**i)
    errors = np.zeros(len(y))
    for j in range(len(y)):
        errors[j] = np.linalg.norm(y[j]-Y(t[j]))
    herr[i][1] = np.max(errors)
    
print("Maximum errors for each h value (formatted as [h,error]):\n" , herr)
print("EOCs for each h value:\n", computeEocs(herr))

maxerrs2 = herr[:,1] #this wil be used in q2.3

Maximum errors for each h value (formatted as [h,error]):
 [[4.00000000e-01 2.33600919e-01]
 [2.00000000e-01 5.72437735e-02]
 [1.00000000e-01 1.25299149e-02]
 [5.00000000e-02 2.89432816e-03]
 [2.50000000e-02 6.88980308e-04]
 [1.25000000e-02 1.68007318e-04]
 [6.25000000e-03 4.14827068e-05]
 [3.12500000e-03 1.03057829e-05]
 [1.56250000e-03 2.56835648e-06]
 [7.81250000e-04 6.41079820e-07]]
EOCs for each h value:
 [2.02885527 2.19174215 2.11407621 2.07069385 2.03593867 2.01794214
 2.00905593 2.00453674 2.00226956]


We showed in Q1.2 that this method converges quadratically, and the EOCs we have calculated reflect that as they are all close to 2, as we expect.

#Q2.3

The only difference between the two programs above is the phi function (either Q11 or forwardEuler). phi is called the same number of times in each program (let's say it is called N times) and so the only difference in the number of times f is called in the number of times each phi function calls f. Looking at the definitions of Q11 and forwardEuler, we see that forwardEuler only calls f once and that Q11 calls f 3 times. Thus, the number of f calls in the first program is N, and the number of calls of f in the second program is 3N

Now, we compare the errors. If both methods were equally efficient with respect to f calls, we would expect that the errors for the Q1.1 method would be about 1/3 of the errors for the forward Euler method. if we were to divide the forward Euler errors by the q1.1 errors, we would expect to get values around 3, however when we do the calulations..

In [17]:
print(maxerrs1/maxerrs2)

[   5.12635169    6.72389792   11.36848979   22.54580791   45.51154538
   91.56444893  183.70389415  368.01450433  736.64524318 1473.91776361]


We see that the values are much larger than 3, especially for very small values of h. To me, this suggests that the Q1.1 method is more efficient with repsect to the number of f calls, as the errors are significantly smaller than the forward Euler method, especially for small values of h

#Q2.4

First let's examine the forward Euler method

In [25]:
e = math.e
pi = math.pi

T = 3
N0 = 25
y0 = array([0])

def f(t, y):
    if t < pi/2:
        return math.cos(t)
    else:
        return y

def Y(t):
    if t < pi/2:
        return math.sin(t)
    else:
        return e**(t-(pi/2))

herr = np.zeros((10,2))

for i in range(10):
    h = T/(N0*2**i)
    herr[i][0] = h
    t,y = evolve( forwardEuler, f, lambda t,y: y, 0, y0, T , N0*2**i)
    errors = np.zeros(len(y))
    for j in range(len(y)):
        errors[j] = np.linalg.norm(y[j]-Y(t[j]))
    herr[i][1] = np.max(errors)

print("Maximum errors for each h value (formatted as [h,error]):\n" , herr)
print("EOCs for each h value:\n", computeEocs(herr))

maxerrs1 = herr[:,1] #this wil be used to compare efficiency

Maximum errors for each h value (formatted as [h,error]):
 [[1.20000000e-01 4.90234371e-01]
 [6.00000000e-02 2.41162142e-01]
 [3.00000000e-02 1.03184451e-01]
 [1.50000000e-02 3.04088602e-02]
 [7.50000000e-03 2.40459232e-02]
 [3.75000000e-03 5.22879930e-03]
 [1.87500000e-03 3.56576540e-03]
 [9.37500000e-04 2.73107305e-03]
 [4.68750000e-04 2.31293004e-03]
 [2.34375000e-04 1.12533445e-03]]
EOCs for each h value:
 [1.02346819 1.22477787 1.76266193 0.33869942 2.20124072 0.55226791
 0.38474388 0.23974627 1.03936779]


From the lectures, we know that the forward Euler method converges linearly (O(h)) to the exact solution, as long as Y is in C^2. However, here the EOCs dont reflect this. This is because Y is not in C^2, as Y is not differentiable at pi/2. Therefore, we cannot be sure that the method converges to the true solution.

In [26]:
herr = np.zeros((10,2))

for i in range(10):
    h = T/(N0*2**i)
    herr[i][0] = h
    t,y = evolve( Q11, f, lambda t,y: y, 0, y0, T , N0*2**i)
    errors = np.zeros(len(y))
    for j in range(len(y)):
        errors[j] = np.linalg.norm(y[j]-Y(t[j]))
    herr[i][1] = np.max(errors)
    
print("Maximum errors for each h value (formatted as [h,error]):\n" , herr)
print("EOCs for each h value:\n", computeEocs(herr))

maxerrs2 = herr[:,1] #this wil be used to compare efficiency

Maximum errors for each h value (formatted as [h,error]):
 [[1.20000000e-01 2.21094332e-01]
 [6.00000000e-02 8.45661338e-02]
 [3.00000000e-02 1.87087291e-02]
 [1.50000000e-02 1.34665651e-02]
 [7.50000000e-03 1.96899368e-03]
 [3.75000000e-03 5.91576066e-03]
 [1.87500000e-03 2.01544743e-03]
 [9.37500000e-04 6.17727487e-05]
 [4.68750000e-04 9.15946526e-04]
 [2.34375000e-04 4.26423985e-04]]
EOCs for each h value:
 [ 1.38651011  2.17636847  0.47432965  2.77385153 -1.58710521  1.55346354
  5.02798581 -3.89022094  1.10297479]


In Q1.2, we showed that if Y is in C^3, then the method converges quadratically, but Y is not in C^3 so we cannot be sure that the method converges to the true solution.

Even though we are not certain the methods converge to the true solution, we can see that the errors still decrease as h decreases, so it is not unreasonable to use them as long as we are careful.

Now, I compare the efficiency of each method for this ODE.

In [27]:
print(maxerrs1/maxerrs2)

[ 2.21730863  2.85175792  5.51531054  2.25810071 12.21229073  0.88387607
  1.76921777 44.21161608  2.52518021  2.63900365]


Recall that from Q1.3, the Q1.1 method uses 3 times as many f calls as the forward Euler, but as we can see, for most vaules of h, the quotient between the errors is less than 3, which suggests that the forward Euler is actually slightly more efficient in terms of f calls than the Q1.1 method for most values of h. However, there are some values of h for which the method converges very quickly (see above where we get 44.2). Thus for some specific values of h, this method is better, but the forward Euler is more consistent overall.